In [ ]:
!pip install tensorflow==2.2.0
import tensorflow as tf
print(tf.__version__)

     |████████████████████████████████| 516.2MB 31kB/s 
     |████████████████████████████████| 3.0MB 18.9MB/s 
  Found existing installation: tensorboard 1.12.2
    Uninstalling tensorboard-1.12.2:
      Successfully uninstalled tensorboard-1.12.2
  Found existing installation: tensorflow 1.12.0
    Uninstalling tensorflow-1.12.0:
      Successfully uninstalled tensorflow-1.12.0


1.12.0


In [1]:
import os
import cv2
import numpy as np
import sys
import time
import tensorflow as tf


In [2]:
! git clone https://github.com/tensorflow/models.git
!apt-get -qq install libprotobuf-java protobuf-compiler
!protoc ./models/research/object_detection/protos/string_int_label_map.proto --python_out=.
!cp -R models/research/object_detection/ object_detection/
!rm -rf models
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile
from collections import defaultdict
from io import StringIO
import matplotlib.pyplot as plt
from PIL import Image
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

Cloning into 'models'...
remote: Enumerating objects: 49, done.
remote: Counting objects: 100% (49/49), done.
remote: Compressing objects: 100% (49/49), done.
remote: Total 38757 (delta 26), reused 23 (delta 0), pack-reused 38708
Receiving objects: 100% (38757/38757), 546.76 MiB | 33.95 MiB/s, done.
Resolving deltas: 100% (25980/25980), done.


In [3]:
from google.colab import files
def upload_files():
  from google.colab import files
  uploaded = files.upload()
  for k, v in uploaded.items():
    open(k, 'wb').write(v)
  return list(uploaded.keys())
upload_files()

Saving sample.mp4 to sample.mp4


['sample.mp4']

In [4]:
import tensorflow.compat.v1 as tf
MODEL_NAME = 'ssd_inception_v2_coco_2017_11_17'
# high accuracy but very slow
# MODEL_NAME = 'faster_rcnn_resnet101_coco_2017_11_08'
MODEL_FILE = MODEL_NAME + '.tar.gz'
DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'
# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_CKPT = MODEL_NAME + '/frozen_inference_graph.pb'
# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = os.path.join('object_detection/data', 'mscoco_label_map.pbtxt')
NUM_CLASSES = 10
opener = urllib.request.URLopener()
opener.retrieve(DOWNLOAD_BASE + MODEL_FILE, MODEL_FILE)
tar_file = tarfile.open(MODEL_FILE)
for file in tar_file.getmembers():
  file_name = os.path.basename(file.name)
  if 'frozen_inference_graph.pb' in file_name:
    tar_file.extract(file, os.getcwd())
    
detection_graph = tf.Graph()
with detection_graph.as_default():
  od_graph_def = tf.compat.v1.GraphDef()
  with tf.io.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
    serialized_graph = fid.read()
    od_graph_def.ParseFromString(serialized_graph)
    tf.import_graph_def(od_graph_def, name='')
    
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)

In [5]:
detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.GraphDef()
    with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')
sess = tf.Session(graph=detection_graph)
# Define input and output tensors (i.e. data) for the object detection classifier
# Input tensor is the image
image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
# Output tensors are the detection boxes, scores, and classes
# Each box represents a part of the image where a particular object was detected
detection_boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
# Each score represents level of confidence for each of the objects.
# The score is shown on the result image, together with the class label.
detection_scores = detection_graph.get_tensor_by_name('detection_scores:0')
detection_classes = detection_graph.get_tensor_by_name('detection_classes:0')
# Number of objects detected
num_detections = detection_graph.get_tensor_by_name('num_detections:0')

In [6]:
def region_of_interest(img, vertices):
    mask = np.zeros_like(img)   
    if len(img.shape) > 2:
        channel_count = img.shape[2]
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255   
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image

In [7]:
video = cv2.VideoCapture("sample.mp4")
fourcc = cv2.VideoWriter_fourcc(*'XVID')
frame_width = int(video.get(3))
frame_height = int(video.get(4))
out = cv2.VideoWriter('outpy.mp4',cv2.VideoWriter_fourcc('M','J','P','G'), 20, (frame_width,frame_height))
try:
  
  while(video.isOpened()):

        ret, frame = video.read()
        stime = time.time()
        objects = []
        class_str = ""
        frame_width = frame.shape[0]
        frame_height = frame.shape[1]
        rows, cols = frame.shape[:2]
        left_boundary = [int(cols*0.40), int(rows*0.95)]
        left_boundary_top = [int(cols*0.40), int(rows*0.20)]
        right_boundary = [int(cols*0.60), int(rows*0.95)]
        right_boundary_top = [int(cols*0.60), int(rows*0.20)]
        bottom_left  = [int(cols*0.20), int(rows*0.95)]
        top_left     = [int(cols*0.20), int(rows*0.20)]
        bottom_right = [int(cols*0.80), int(rows*0.95)]
        top_right    = [int(cols*0.80), int(rows*0.20)]

        
        vertices = np.array([[bottom_left, top_left, top_right, bottom_right]], dtype=np.int32)
        cv2.line(frame,tuple(bottom_left),tuple(bottom_right), (255, 0, 0), 5)
        cv2.line(frame,tuple(bottom_right),tuple(top_right), (255, 0, 0), 5)
        cv2.line(frame,tuple(top_left),tuple(bottom_left), (255, 0, 0), 5)
        cv2.line(frame,tuple(top_left),tuple(top_right), (255, 0, 0), 5)
        copied = np.copy(frame)
        interested=region_of_interest(copied,vertices)
        frame_expanded = np.expand_dims(interested, axis=0)

        (boxes, scores, classes, num) = sess.run(
            [detection_boxes, detection_scores, detection_classes, num_detections],
            feed_dict={image_tensor: frame_expanded})
        vis_util.visualize_boxes_and_labels_on_image_array(
            frame,
            np.squeeze(boxes),
            np.squeeze(classes).astype(np.int32),
            np.squeeze(scores),
            category_index,
            use_normalized_coordinates=True,
            line_thickness=8,
            min_score_thresh=0.78)
        print(frame_width,frame_height)
        ymin = int((boxes[0][0][0]*frame_width))
        xmin = int((boxes[0][0][1]*frame_height))
        ymax = int((boxes[0][0][2]*frame_width))
        xmax = int((boxes[0][0][3]*frame_height))
        Result = np.array(frame[ymin:ymax,xmin:xmax])

        ymin_str='y min  = %.2f '%(ymin)
        ymax_str='y max  = %.2f '%(ymax)
        xmin_str='x min  = %.2f '%(xmin)
        xmax_str='x max  = %.2f '%(xmax)

        cv2.putText(frame,ymin_str, (50, 50),cv2.FONT_HERSHEY_SIMPLEX,0.6,(255,0,0),2)
        cv2.putText(frame,ymax_str, (50, 70),cv2.FONT_HERSHEY_SIMPLEX,0.6,(255,0,0),2)
        cv2.putText(frame,xmin_str, (50, 90),cv2.FONT_HERSHEY_SIMPLEX,0.6,(255,0,0),2)
        cv2.putText(frame,xmax_str, (50, 110),cv2.FONT_HERSHEY_SIMPLEX,0.6,(255,0,0),2)
        print(scores.max())
      
        print("left_boundary[0],right_boundary[0] :", left_boundary[0], right_boundary[0])
        print("left_boundary[1],right_boundary[1] :", left_boundary[1], right_boundary[1])
        print("xmin, xmax :", xmin, xmax)
        print("ymin, ymax :", ymin, ymax)
        if scores.max() > 0.78:
          print("inif")
        if(xmin >= left_boundary[0]):
            print("move LEFT - 1st !!!")
            cv2.putText(frame,'Move LEFT!', (300, 100),cv2.FONT_HERSHEY_SIMPLEX,1.5,(0,255,0),2)
        elif(xmax <= right_boundary[0]):
            print("move Right - 2nd !!!")
            cv2.putText(frame,'Move RIGHT!', (300, 100),cv2.FONT_HERSHEY_SIMPLEX,1.5,(0,255,0),2)
        elif(xmin <= left_boundary[0] and xmax >= right_boundary[0]):
            print("STOPPPPPP !!!! - 3nd !!!")
            cv2.putText(frame,' STOPPPPPP!!!', (300, 100),cv2.FONT_HERSHEY_SIMPLEX,1.5,(0,255,0),2)
            cv2.line(frame,tuple(left_boundary),tuple(left_boundary_top), (255, 0, 0), 5)
        cv2.line(frame,tuple(right_boundary),tuple(right_boundary_top), (255, 0, 0), 5)
  out.write(frame)
except:
  pass

352 640
0.20763373
left_boundary[0],right_boundary[0] : 256 384
left_boundary[1],right_boundary[1] : 334 334
xmin, xmax : 141 219
ymin, ymax : 188 205
move Right - 2nd !!!
352 640
0.21202046
left_boundary[0],right_boundary[0] : 256 384
left_boundary[1],right_boundary[1] : 334 334
xmin, xmax : 144 218
ymin, ymax : 188 205
move Right - 2nd !!!
352 640
0.16892192
left_boundary[0],right_boundary[0] : 256 384
left_boundary[1],right_boundary[1] : 334 334
xmin, xmax : 247 260
ymin, ymax : 155 188
move Right - 2nd !!!
352 640
0.1625295
left_boundary[0],right_boundary[0] : 256 384
left_boundary[1],right_boundary[1] : 334 334
xmin, xmax : 137 223
ymin, ymax : 198 222
move Right - 2nd !!!
352 640
0.20235837
left_boundary[0],right_boundary[0] : 256 384
left_boundary[1],right_boundary[1] : 334 334
xmin, xmax : 145 213
ymin, ymax : 187 204
move Right - 2nd !!!
352 640
0.13623574
left_boundary[0],right_boundary[0] : 256 384
left_boundary[1],right_boundary[1] : 334 334
xmin, xmax : 212 511
ymin, ymax 